# Train/Test Split for Horse Detection Dataset

This notebook implements a train/test split for the horse detection dataset according to the following requirements:

1. Reserve 20% of the data for final evaluation
2. For absences: exclude any absence found within 2 meters of a presence (at any time point), then group by spatial blocks and sample 20% randomly from each group
3. For presences: group by zone and period, sampling 20% randomly from each group
4. Save each split in its own file and document in the Hugging Face markdown file

## Setup

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
from pathlib import Path
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
import random
import json

# Set random seed for reproducibility
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

## Load Ground Truth Data

In [ ]:
# Load ground truth data
groundtruth_path = '../data/vector/groundtruth.geojson'
gdf = gpd.read_file(groundtruth_path)

# Display basic information
print(f"Total points: {len(gdf)}")
print(f"Presence points: {len(gdf[gdf['Presence'] == 1])}")
print(f"Absence points: {len(gdf[gdf['Presence'] == 0])}")
print(f"Unique zones: {gdf['Zone'].unique()}")
print(f"Unique periods: {gdf['Period'].unique()}")

## Create Spatial Blocks

We'll use K-means clustering to create 10 spatial blocks based on the coordinates.

In [ ]:
# Extract coordinates for clustering
coords = np.column_stack((gdf.geometry.x, gdf.geometry.y))

# Create 10 spatial blocks using K-means clustering
kmeans = KMeans(n_clusters=10, random_state=RANDOM_SEED)
gdf['spatial_block'] = kmeans.fit_predict(coords)

# Visualize the spatial blocks
plt.figure(figsize=(12, 10))
for block_id in range(10):
    block_points = gdf[gdf['spatial_block'] == block_id]
    plt.scatter(block_points.geometry.x, block_points.geometry.y, 
                label=f'Block {block_id}', alpha=0.7)

plt.title('Spatial Blocks Created by K-means Clustering')
plt.xlabel('Easting')
plt.ylabel('Northing')
plt.legend()
plt.grid(True)
plt.show()

# Count points in each spatial block
block_counts = gdf.groupby('spatial_block').size()
print("Points per spatial block:")
print(block_counts)

## Process Absences

1. Exclude absences within 2 meters of any presence
2. Group by spatial block
3. Sample 20% from each group for testing

In [ ]:
# Separate presence and absence points
presence_gdf = gdf[gdf['Presence'] == 1]
absence_gdf = gdf[gdf['Presence'] == 0]

print(f"Original absence points: {len(absence_gdf)}")

# Create a 2-meter buffer around each presence point
presence_buffers = presence_gdf.copy()
presence_buffers.geometry = presence_gdf.geometry.buffer(2)

# Find absences that are NOT within any presence buffer
valid_absences = absence_gdf.copy()
for idx, buffer in tqdm(presence_buffers.iterrows(), total=len(presence_buffers), desc="Filtering absences"):
    # Remove absences that intersect with this buffer
    valid_absences = valid_absences[~valid_absences.geometry.intersects(buffer.geometry)]

print(f"Valid absence points (after removing those within 2m of presences): {len(valid_absences)}")

# Group valid absences by spatial block
absence_test_indices = []
absence_train_indices = []

for block_id in valid_absences['spatial_block'].unique():
    block_absences = valid_absences[valid_absences['spatial_block'] == block_id]
    
    # Sample 20% for testing
    test_size = max(1, int(0.2 * len(block_absences)))
    test_indices = block_absences.sample(n=test_size, random_state=RANDOM_SEED).index.tolist()
    train_indices = block_absences.index.difference(test_indices).tolist()
    
    absence_test_indices.extend(test_indices)
    absence_train_indices.extend(train_indices)

print(f"Absence points for testing: {len(absence_test_indices)}")
print(f"Absence points for training: {len(absence_train_indices)}")

## Process Presences

1. Group by zone and period
2. Sample 20% from each group for testing

In [ ]:
# Group presences by zone and period
presence_test_indices = []
presence_train_indices = []

for zone in presence_gdf['Zone'].unique():
    for period in presence_gdf['Period'].unique():
        group = presence_gdf[(presence_gdf['Zone'] == zone) & (presence_gdf['Period'] == period)]
        
        if len(group) == 0:
            continue
        
        # Sample 20% for testing
        test_size = max(1, int(0.2 * len(group)))
        test_indices = group.sample(n=test_size, random_state=RANDOM_SEED).index.tolist()
        train_indices = group.index.difference(test_indices).tolist()
        
        presence_test_indices.extend(test_indices)
        presence_train_indices.extend(train_indices)

print(f"Presence points for testing: {len(presence_test_indices)}")
print(f"Presence points for training: {len(presence_train_indices)}")

## Combine and Save Splits

In [ ]:
# Combine test and train indices
test_indices = presence_test_indices + absence_test_indices
train_indices = presence_train_indices + absence_train_indices

# Create test and train dataframes
test_gdf = gdf.loc[test_indices].copy()
train_gdf = gdf.loc[train_indices].copy()

# Verify the split
print(f"Total points: {len(gdf)}")
print(f"Training points: {len(train_gdf)} ({len(train_gdf)/len(gdf)*100:.1f}%)")
print(f"Testing points: {len(test_gdf)} ({len(test_gdf)/len(gdf)*100:.1f}%)")

# Check class distribution in each split
print("\nClass distribution in training set:")
print(train_gdf['Presence'].value_counts(normalize=True) * 100)

print("\nClass distribution in test set:")
print(test_gdf['Presence'].value_counts(normalize=True) * 100)

## Visualize the Split

In [ ]:
# Visualize the train/test split
plt.figure(figsize=(12, 10))

# Plot training points
plt.scatter(train_gdf[train_gdf['Presence'] == 1].geometry.x, 
            train_gdf[train_gdf['Presence'] == 1].geometry.y, 
            c='blue', marker='^', alpha=0.7, label='Train - Presence')
plt.scatter(train_gdf[train_gdf['Presence'] == 0].geometry.x, 
            train_gdf[train_gdf['Presence'] == 0].geometry.y, 
            c='lightblue', marker='o', alpha=0.7, label='Train - Absence')

# Plot testing points
plt.scatter(test_gdf[test_gdf['Presence'] == 1].geometry.x, 
            test_gdf[test_gdf['Presence'] == 1].geometry.y, 
            c='red', marker='^', alpha=0.7, label='Test - Presence')
plt.scatter(test_gdf[test_gdf['Presence'] == 0].geometry.x, 
            test_gdf[test_gdf['Presence'] == 0].geometry.y, 
            c='pink', marker='o', alpha=0.7, label='Test - Absence')

plt.title('Train/Test Split Visualization')
plt.xlabel('Easting')
plt.ylabel('Northing')
plt.legend()
plt.grid(True)
plt.show()

## Save the Splits

In [ ]:
# Create output directory
output_dir = '../data/processed/splits'
os.makedirs(output_dir, exist_ok=True)

# Save the GeoDataFrames
train_gdf.to_file(os.path.join(output_dir, 'train_split.geojson'), driver='GeoJSON')
test_gdf.to_file(os.path.join(output_dir, 'test_split.geojson'), driver='GeoJSON')

# Save the indices as JSON for reference
split_indices = {
    'train_indices': train_indices,
    'test_indices': test_indices,
    'metadata': {
        'random_seed': RANDOM_SEED,
        'train_size': len(train_indices),
        'test_size': len(test_indices),
        'train_presence_count': len(train_gdf[train_gdf['Presence'] == 1]),
        'train_absence_count': len(train_gdf[train_gdf['Presence'] == 0]),
        'test_presence_count': len(test_gdf[test_gdf['Presence'] == 1]),
        'test_absence_count': len(test_gdf[test_gdf['Presence'] == 0]),
    }
}

with open(os.path.join(output_dir, 'split_indices.json'), 'w') as f:
    json.dump(split_indices, f, indent=2)

print(f"Splits saved to {output_dir}")

## Create Hugging Face Dataset Documentation

In [ ]:
# Create Markdown documentation for Hugging Face
hf_markdown = f"""
# MPG Ranch Horse Detection Dataset - Train/Test Split

## Dataset Description

This dataset contains aerial imagery tiles for horse presence/absence detection at MPG Ranch. 
The data has been split into training and testing sets according to specific criteria.

## Split Methodology

The dataset was split into training (80%) and testing (20%) sets using the following criteria:

1. **Random Seed**: {RANDOM_SEED} (for reproducibility)
2. **Absence Points**: 
   - Excluded any absence found within 2 meters of a presence (at any time point)
   - Grouped by 10 spatial blocks created using K-means clustering
   - Sampled 20% randomly from each spatial block for testing
3. **Presence Points**:
   - Grouped by zone and period
   - Sampled 20% randomly from each group for testing

## Split Statistics

- **Total Points**: {len(gdf)}
- **Training Set**: {len(train_gdf)} points ({len(train_gdf)/len(gdf)*100:.1f}%)
  - Presence: {len(train_gdf[train_gdf['Presence'] == 1])}
  - Absence: {len(train_gdf[train_gdf['Presence'] == 0])}
- **Testing Set**: {len(test_gdf)} points ({len(test_gdf)/len(gdf)*100:.1f}%)
  - Presence: {len(test_gdf[test_gdf['Presence'] == 1])}
  - Absence: {len(test_gdf[test_gdf['Presence'] == 0])}

## Files

- `train_split.geojson`: GeoJSON file containing the training points
- `test_split.geojson`: GeoJSON file containing the testing points
- `split_indices.json`: JSON file containing the indices of the training and testing points

## Usage

```python
import geopandas as gpd

# Load the splits
train_gdf = gpd.read_file('train_split.geojson')
test_gdf = gpd.read_file('test_split.geojson')
```
"""

# Save the markdown file
with open(os.path.join(output_dir, 'README.md'), 'w') as f:
    f.write(hf_markdown)

print("Hugging Face documentation created.")